In [ ]:
import pandas as pd
import numpy as np
from google.cloud.bigquery import Client, QueryJobConfig
import seaborn as sns
import os

In [ ]:
from google.cloud.bigquery import Client, QueryJobConfig
client = Client()

In [ ]:
query = """
SELECT 
  fo_id.yeti_customer_uuid,
  #MIN(fo_id.first_order_id) as first_order_id, 
  #COUNT(oi_prod_table.product_id) as num_prods_in_first_order, 
  #COUNT(DISTINCT oi_prod_table.product_category) as num_distinct_prod_cats, 
  MAX(oi_prod_table.price) as max_entry_product_price,
FROM
  (SELECT
    orders.yeti_customer_uuid,
    COUNT(orders.order_id) as order_cnt,
    MIN(orders.order_id) as first_order_id,
    MIN(orders.creation_date) first_order_date,
  FROM `yeti-dev-edw.aa_anon.aa_order` orders
  WHERE orders.source_country = 'US' AND orders.site_id = 'Yeti_US'
  GROUP BY orders.yeti_customer_uuid
  HAVING order_cnt > 1) fo_id
LEFT JOIN
  (
  SELECT
    order_item.yeti_customer_uuid,
    order_item.order_id,
    order_item.order_item_id,
    order_item.product_id, 
    product.product_category, 
    product.price, 
  FROM `yeti-dev-edw.aa_anon.aa_order_item` order_item
  LEFT JOIN `yeti-dev-edw.aa_anon.aa_product` product
  ON order_item.product_id = product.product_id) oi_prod_table
ON (fo_id.yeti_customer_uuid = oi_prod_table.yeti_customer_uuid AND fo_id.first_order_id = oi_prod_table.order_id)
GROUP BY yeti_customer_uuid
"""

job = client.query(query)
df = job.to_dataframe()

In [ ]:
df

,yeti_customer_uuid,max_entry_product_price
0,92ca2474-0108-4e55-842f-7c7f8849db18,65.00
1,c2a3c6bd-8124-484d-a561-aa2d9999e5c6,3.00
2,336cd0de-4aa9-4252-961e-06a6d1de13b1,9.00
3,fb0791d5-491a-4d28-8aaf-bbd579d99092,39.99
4,acbbf37d-ad48-430c-8f24-c4ddbabbe144,17.00
...,...,...
2131516,fb844946-773b-4f19-a11c-3790b49f7f0a,14.99
2131517,f2bed748-8f8a-4ed0-858d-7b811ab72a53,14.99
2131518,56b2ec66-40a2-4dcd-9bcd-dc61a080e96c,14.99
2131519,b3bed4a1-9279-4a49-9696-ab2c079c919d,14.99


In [ ]:
df.to_csv('/home/jupyter/Data_files/data_entry_prod_price.csv')

In [ ]:
os.getcwd()

'/home/jupyter/Data Engineering'

### Combining this with latest data

In [ ]:
data_entry_prod_price = pd.read_csv('/home/jupyter/Data_files/data_entry_prod_price.csv')

In [ ]:
data_entry_prod_price = data_entry_prod_price.iloc[:,1:]

In [ ]:
data_drop_outliers = pd.read_csv('/home/jupyter/Data_files/data_drop_outlier_03_17_23.csv')

In [ ]:
data_drop_outliers = data_drop_outliers.iloc[:,2:]

In [ ]:
data_merged = pd.merge(data_drop_outliers, data_entry_prod_price, how='left',left_on = 'customer_id',right_on = 'yeti_customer_uuid')

In [ ]:
data_merged = data_merged.drop(columns = ['yeti_customer_uuid','entry_prod_cat'])

In [ ]:
data_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1706046 entries, 0 to 1706045
Data columns (total 41 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   customer_id                      object 
 1   avg_time_btw_orders              float64
 2   recency                          float64
 3   months_elapsed                   float64
 4   purchase_times                   int64  
 5   lifetime_spent                   float64
 6   lifetime_quantity                float64
 7   per_order_spent                  float64
 8   per_order_quantity               float64
 9   distinct_category_count          int64  
 10  discount_frequency_order         float64
 11  discount_frequency_product       float64
 12  outdoor_equipment_pct            float64
 13  cargo_pct                        float64
 14  soft_cooler_pct                  float64
 15  bags_pct                         float64
 16  drinkware_pct                    float64
 17  hard_coo

### Export this csv

In [ ]:
data_merged.to_csv('/home/jupyter/Data_files/data_w_entry_prod_price_no_outliers_03_22_23.csv')